##### Downloads - only to it once

In [ ]:
# Installing packages
# non-conda packages
! pip install twitter
! pip install -U nltk
! pip install -U textblob
! pip install networkx
! pip install gender-guesser

# conda packages
! conda install -c conda-forge -y wordcloud
    
# additional downloads    
! python -m textblob.download_corpora
! python -m nltk.downloader stopwords

##### Import modules

In [17]:
# Import modules 
import codecs
from collections import Counter
from datetime import date, datetime, timedelta, timezone
import pickle
import pytz
import re

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import networkx as nx
import nltk
from textblob import TextBlob
import twitter
from wordcloud import WordCloud

%matplotlib inline

In [6]:
# hrep twitters
df = pd.read_excel('rep_twitters.xlsx')

In [8]:
hrep_twhandles = list(df.tw_name)

In [11]:
with open('hrep_twhandles.txt', 'w', encoding='utf-8') as f:
    f.write(str(hrep_twhandles))

In [12]:
df1 = pd.read_excel('sen_list_116.xlsx')

In [13]:
df1.head(3)

,name,screen_name
0,Hawley Press Office,@SenHawleyPress
1,Mike Braun,@SenatorBraun
2,Mitt Romney,@SenatorRomney


In [15]:
sen_twhandles = list(df1.screen_name)

In [16]:
with open('sen_twhandles.txt', 'w', encoding='utf-8') as f:
    f.write(str(sen_twhandles))

##### Functions 

In [2]:
# clean up text 
def cleanse_text(text):
    text = text.replace("\n", "")
    text = text.replace("\r", "")
    text = text.replace("\t", "")
    text = text.replace("\"", "")
    
    return text

def save_latest_id(string):
    with open('latest_id.txt', 'wb') as f:
        pickle.dump(string, f)
        
def load_latest_id(pickle_file):
    with open('latest_id.txt', 'rb') as f:
        data = pickle.load(f)
        
        return data

##### Connect to the Twitter API

In [3]:
# parameters 
CONSUMER_KEY = "LH4kuDFYDbitBOHXX0dAiXjUk"
CONSUMER_SECRET = "wiucSGOJkhhhUqtMgvWKjuengTMvPDVDksluyTqUo5YEVpDxWh"
ACCESS_TOKEN = "166588006-si4UHhFeVgMuo3yxBXMGbsJFY5tmeRngkjgbmbMX"
ACCESS_TOKEN_SECRET = "vawC4Ui5sj4RqbDpwA6fsjM5fOSPLYxccHWXMkvUtKZBK"

In [4]:
# Make a connectino to Twitter 
auth = twitter.oauth.OAuth(ACCESS_TOKEN, ACCESS_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
twitter_api = twitter.Twitter(auth=auth)

##### Twitter Search - User Timeline

In [38]:
screen_name = "RepAOC"
search_date = datetime.now(pytz.utc)

In [42]:
target_week = search_date - timedelta(days=7)

In [6]:
timeline_result = twitter_api.statuses.user_timeline(screen_name="RepAOC")

In [7]:
len(timeline_result)

15

In [8]:
timeline_result[1].keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang'])

In [9]:
save_latest_id(timeline_result[0]['id_str'])

In [10]:
# data to CSV 
file = f"search_data_{screen_name}_{search_date}.csv"
with open(file=file, mode="w", encoding="utf8") as fw:
    fw.write("id\t" + 
             "created_at\t" + 
             "text\t" + 
             "hashtags\t" + 
             "symbols\t" +
             "user_mentions\t" + 
             "user_id\t" + 
             "user_name\t" + 
             "user_screen_name\t" + 
             "user_followers_count\t" + 
             "retweet_count\t" + 
             "favorite_count\n")
    for tweet in timeline_result:
        fw.write(tweet['id_str'] + '\t')
        fw.write(tweet['created_at'] + '\t')
        fw.write(cleanse_text(tweet['text']) + '\t')
        fw.write(str(tweet['entities']['hashtags']) + '\t')
        fw.write(str(tweet['entities']['symbols']) + '\t')
        fw.write(str(tweet['entities']['user_mentions']) + '\t')
        fw.write(tweet['user']['id_str'] + '\t')
        fw.write(tweet['user']['name'] + '\t')
        fw.write(tweet['user']['screen_name'] + '\t')
        fw.write(str(tweet['user']['followers_count']) + '\t')
        fw.write(str(tweet['retweet_count']) + '\t')
        fw.write(str(tweet['favorite_count']) + '\t')
        fw.write("\n")

In [11]:
# load the csv file as a pandas df
df = pd.read_csv(file, sep="\t", header=0, parse_dates=[1], index_col=False)

In [12]:
df.head()

,id,created_at,text,hashtags,symbols,user_mentions,user_id,user_name,user_screen_name,user_followers_count,retweet_count,favorite_count
0,1096112733400035328,2019-02-14 18:23:35+00:00,A joint statement from the offices of @Ilhan @...,[],[],"[{'screen_name': 'Ilhan', 'name': 'Rep. Ilhan ...",1079104563280527364,Rep. Alexandria Ocasio-Cortez,RepAOC,124513,398,1397
1,1093582852917219337,2019-02-07 18:50:44+00:00,Proud to stand with @SenMarkey and over 60 Ori...,"[{'text': 'GreenNewDeal', 'indices': [92, 105]}]",[],"[{'screen_name': 'SenMarkey', 'name': 'Ed Mark...",1079104563280527364,Rep. Alexandria Ocasio-Cortez,RepAOC,124513,1380,4560
2,1090471469170245637,2019-01-30 04:47:12+00:00,RT @RepRashida: Today was my first hearing on ...,[],[],"[{'screen_name': 'RepRashida', 'name': 'Congre...",1079104563280527364,Rep. Alexandria Ocasio-Cortez,RepAOC,124513,2212,0
3,1090471337724964870,2019-01-30 04:46:41+00:00,Today @OversightDems held its first-ever heari...,[],[],"[{'screen_name': 'OversightDems', 'name': 'Ove...",1079104563280527364,Rep. Alexandria Ocasio-Cortez,RepAOC,124513,349,2697
4,1088209102722031616,2019-01-23 22:57:22+00:00,The post-its outside our office are growing 💜 ...,[],[],[],1079104563280527364,Rep. Alexandria Ocasio-Cortez,RepAOC,124513,477,5267


##### Tokenization & tagging 

In [ ]:
df['tags'] = df.text.apply(lambda x:TextBlob(x).tags)

In [ ]:
df[['text', 'tags']].head()

In [ ]:
df['tokens'] = df.text.apply(lambda x:x.split(' '))

In [ ]:
df[['text', 'tags', 'tokens']].head()

##### Frequency

In [68]:
global_stopwords = nltk.corpus.stopwords.words("english")
local_stopwords = ["@", "&", "‘", "’", "“", "”", "…", "'", "", "'s", "''"]

In [ ]:
counter = Counter()

for tags in df['tags']:
    word_set = set()
    
    for item in tags:
        word = item[0].lower()
        
        if word in (global_stopwords + local_stopwords):
            continue
        else:
            word_set.add(word)
            
    counter.update(word_set)

counter.most_common(10)

In [ ]:
for tokens in df['tokens']:
    tokens_set = set()
    
    for token in tokens:
        token = token.lower()
        
        if token in (global_stopwords + local_stopwords):
            continue
        else:
            tokens_set.add(token)
    
    counter.update(tokens_set)
        
counter.most_common(10)

##### For continuous data collecting

In [ ]:
screen_name = "RepAOC"
search_date = date.today().strftime("%m-%d-%y")

In [ ]:
timeline_result = twitter_api.statuses.user_timeline(screen_name="RepAOC", 
                                                     since_id=load_latest_id('latest_id.txt'),
                                                     count='100')

##### Twitter timelines for multiple screen names

In [13]:
# bring in Rep Twitter data
df_rep_twitters = pd.read_excel('rep_twitters.xlsx', encoding='utf-8')

In [14]:
screen_names = df_rep_twitters['tw_name'].to_list()

In [15]:
len(screen_names)

439

In [16]:
# creating DF for result data
df_twitter_results = pd.DataFrame(columns=['id_str', 'created_at', 'text', 'hashtags', 'symbols', 'user_mentions',
                                           'user_id', 'user_name', 'user_screen_name', 'user_followers_count', 
                                           'retweet_count', 'favorite_count'])

In [17]:
dict_twitter_results = {'id_str': [], 
                       'created_at': [], 
                       'text': [], 
                       'hashtags': [], 
                       'symbols': [], 
                       'user_mentions': [],
                       'user_id': [], 
                       'user_name': [], 
                       'user_screen_name': [], 
                       'user_followers_count': [], 
                       'retweet_count': [], 
                       'favorite_count': []}

In [18]:
for item in screen_names:
    t_result = twitter_api.statuses.user_timeline(screen_name=item, count=20)
    for tweet in t_result:
        dict_twitter_results['id_str'].append(tweet['id_str'])
        dict_twitter_results['created_at'].append(tweet['created_at'])
        dict_twitter_results['text'].append(tweet['text'])
        dict_twitter_results['hashtags'].append(str(tweet['entities']['hashtags']))
        dict_twitter_results['symbols'].append(str(tweet['entities']['symbols']))
        dict_twitter_results['user_mentions'].append(str(tweet['entities']['user_mentions']))
        dict_twitter_results['user_id'].append(tweet['user']['id_str'])
        dict_twitter_results['user_name'].append(tweet['user']['name'])
        dict_twitter_results['user_screen_name'].append(tweet['user']['screen_name'])
        dict_twitter_results['user_followers_count'].append(str(tweet['user']['followers_count']))
        dict_twitter_results['retweet_count'].append(str(tweet['retweet_count']))
        dict_twitter_results['favorite_count'].append(str(tweet['favorite_count']))

In [19]:
df_tw = pd.DataFrame.from_dict(dict_twitter_results)

In [27]:
s = df_tw['created_at']
s2 = pd.to_datetime(s, infer_datetime_format=True)
s2[0]

Timestamp('2019-02-18 16:30:00+0000', tz='UTC')

In [34]:
df_tw['post_date'] = s2

In [36]:
df_tw.head(2)

,id_str,created_at,text,hashtags,symbols,user_mentions,user_id,user_name,user_screen_name,user_followers_count,retweet_count,favorite_count,post_date
0,1097533701775187968,Mon Feb 18 16:30:00 +0000 2019,"“If the Green New Deal becomes a reality, the ...",[],[],[],2962891515,Rep. Ralph Abraham,RepAbraham,16026,4,10,2019-02-18 16:30:00+00:00
1,1097190187942043648,Sun Feb 17 17:45:00 +0000 2019,"If we don’t stand for life, who will? While ot...",[],[],[],2962891515,Rep. Ralph Abraham,RepAbraham,16026,8,25,2019-02-17 17:45:00+00:00


In [43]:
df_tw.to_csv('twitter_results.csv', encoding='utf-8')

In [48]:
df_tw_week = df_tw[(df_tw['post_date'] > target_week)]

In [49]:
df_tw_week['tags'] = df_tw_week['text'].apply(lambda x: TextBlob(x).tags)

/root/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
df_tw_week[['text', 'tags']].head(2)

,text,tags
0,"“If the Green New Deal becomes a reality, the ...","[(“, NN), (If, IN), (the, DT), (Green, NNP), (..."
1,"If we don’t stand for life, who will? While ot...","[(If, IN), (we, PRP), (don, VBP), (’, JJ), (t,..."


In [58]:
df_tw_week['tokens'] = df_tw_week['text'].apply(lambda x: re.sub('[^\w\s]', '', x).split(' '))

/root/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [73]:
df_tw_week['retweet'] = df_tw_week['tokens'].apply(lambda x: 1 if x[0] == 'RT' else 0)

/root/miniconda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [74]:
df_tw_week[['text', 'tags', 'tokens', 'retweet']].head(50)

,text,tags,tokens,retweet
0,"“If the Green New Deal becomes a reality, the ...","[(“, NN), (If, IN), (the, DT), (Green, NNP), (...","[If, the, Green, New, Deal, becomes, a, realit...",0
1,"If we don’t stand for life, who will? While ot...","[(If, IN), (we, PRP), (don, VBP), (’, JJ), (t,...","[If, we, dont, stand, for, life, who, will, Wh...",0
2,Anyone who thinks our crisis at the border is ...,"[(Anyone, NN), (who, WP), (thinks, VBZ), (our,...","[Anyone, who, thinks, our, crisis, at, the, bo...",0
3,“Republican Rep. Ralph Abraham isn't going to ...,"[(“, JJ), (Republican, NNP), (Rep., NNP), (Ral...","[Republican, Rep, Ralph, Abraham, isnt, going,...",0
4,Thank you to @WashTimes for allowing @SenJohnK...,"[(Thank, NNP), (you, PRP), (to, TO), (@, VB), ...","[Thank, you, to, WashTimes, for, allowing, Sen...",0
5,I’m thankful to have @SecretarySonny’s support...,"[(I, PRP), (’, VBP), (m, RB), (thankful, JJ), ...","[Im, thankful, to, have, SecretarySonnys, supp...",0
6,"This week, we salute the more than 9 million v...","[(This, DT), (week, NN), (we, PRP), (salute, V...","[This, week, we, salute, the, more, than, 9, m...",0
7,If President Trump needs to declare a #Nationa...,"[(If, IN), (President, NNP), (Trump, NNP), (ne...","[If, President, Trump, needs, to, declare, a, ...",0
8,I voted NO. This bill limits the number of ill...,"[(I, PRP), (voted, VBD), (NO, NNP), (This, DT)...","[I, voted, NO, This, bill, limits, the, number...",0
9,"As a pilot, I’ve seen firsthand – from the coc...","[(As, IN), (a, DT), (pilot, NN), (I, PRP), (’,...","[As, a, pilot, Ive, seen, firsthand, , from, t...",0


In [70]:
counter = Counter()

for tags in df_tw_week['tags']:
    word_set = set()
    
    for item in tags:
        word = item[0].lower()
        
        if word in (global_stopwords + local_stopwords):
            continue
        else:
            word_set.add(word)
            
    counter.update(word_set)

counter.most_common(10)

[('https', 4371),
 ('rt', 1249),
 ('today', 658),
 ('amp', 460),
 ('president', 430),
 ('national', 305),
 ('congress', 300),
 ('emergency', 286),
 ('trump', 279),
 ('bill', 256)]

In [69]:
for tokens in df_tw_week['tokens']:
    tokens_set = set()
    
    for token in tokens:
        token = token.lower()
        
        if token in (global_stopwords + local_stopwords):
            continue
        else:
            tokens_set.add(token)
    
    counter.update(tokens_set)
        
counter.most_common(50)

[('rt', 6249),
 ('https', 4391),
 ('today', 3070),
 ('amp', 2256),
 ('president', 1830),
 ('national', 1549),
 ('congress', 1488),
 ('emergency', 1382),
 ('bill', 1264),
 ('border', 1239),
 ('', 1160),
 ('house', 1132),
 ('great', 1125),
 ('one', 1093),
 ('us', 1084),
 ('act', 1083),
 ('thank', 1070),
 ('trump', 1055),
 ('new', 951),
 ('bipartisan', 946),
 ('happy', 926),
 ('people', 926),
 ('day', 916),
 ('year', 904),
 ('work', 876),
 ('im', 860),
 ('proud', 841),
 ('time', 804),
 ('support', 778),
 ('first', 768),
 ('families', 746),
 ('parkland', 745),
 ('community', 739),
 ('would', 733),
 ('need', 733),
 ('gun', 726),
 ('week', 706),
 ('last', 705),
 ('honor', 701),
 ('american', 700),
 ('security', 658),
 ('government', 653),
 ('presidents', 625),
 ('wall', 620),
 ('violence', 619),
 ('funding', 608),
 ('discuss', 604),
 ('join', 571),
 ('deal', 552),
 ('like', 540)]

### Members of House of Rep. and Their Twitter Names
### No need to process below this cell.. 

##### list of all memebers of House of Representatives

In [ ]:
# List of HR members 
df_rep_members = pd.read_csv('US_House_116.csv', encoding='utf-8')

In [ ]:
df_rep_members = df_rep_members[['Name', 'District']]

In [ ]:
df_rep_members['last_name'] = df_hrep_members['Name'].apply(lambda x: x.split(', ')[0])

In [ ]:
df_rep_members.head(10)

In [ ]:
df_rep_members.to_csv('output1.csv')

In [ ]:
# List of Twitter handles 
df_rep_twhandles = pd.read_csv('hrep_twitter_names.csv', encoding='utf-8')

In [ ]:
df_rep_twhandles['tw_handle'] = df_rep_twhandles['List members'].apply(lambda x: x.split('Verified'))

In [ ]:
group_handles = []
df_rep_twhandles['name'] = np.nan
df_rep_twhandles['tw_name'] = np.nan
for idx in range(0, len(df_rep_twhandles['tw_handle'])):
    df_rep_twhandles['name'][idx] = df_rep_twhandles['tw_handle'][idx][0].strip()
    
    if len(df_rep_twhandles['tw_handle'][idx]) < 2:
        group_handles.append(df_rep_twhandles['tw_handle'][idx][0])
        continue
    else:
        df_rep_twhandles['tw_name'][idx] = "@" + df_rep_twhandles['tw_handle'][idx][1].strip().split('@')[-1]

In [ ]:
df_rep_twnames = df_rep_twhandles[['name', 'tw_name']]

In [ ]:
df_rep_twnames.head()

In [ ]:
df_rep_twnames['last_name'] = df_rep_twnames['name'].apply(lambda x: x.split(' ')[-1])

In [ ]:
df_rep_twnames.to_csv('output2.csv', encoding='utf-8')

In [ ]:
df_rep_twnames['name'] = df_rep_twnames['name'].apply(lambda x: x.replace('Congressman', ''))
df_rep_twnames['name'] = df_rep_twnames['name'].apply(lambda x: x.replace('Congresswoman', ''))
df_rep_twnames['name'] = df_rep_twnames['name'].apply(lambda x: x.replace('Rep.', ''))

In [ ]:
df_rep_twnames.head(10)

In [ ]:
df_rep = pd.merge(df_rep_members, df_rep_twnames, on=['last_name'], how='outer')

In [ ]:
df_rep.to_excel('output3.xlsx', encoding='utf-8')